In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import librosa as li
import time

from IPython import display
from concurrent.futures import ThreadPoolExecutor

%matplotlib inline

print("TensorFlow version is ", tf.__version__)

TensorFlow version is  2.3.1


In [2]:
PATH_TO_MIRA = 'AudioData/5 Мира'

paths = li.util.find_files(PATH_TO_MIRA)
print(len(paths))
print('\n'.join(paths[:10]))

70
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\1.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\10.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\11.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\12.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\13.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\14.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\15.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\16.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\17.wav
F:\TF\AudioRec\AudioData\5 Мира\1 Мира\18.wav


In [3]:
def get_audio(path, sr=16000):
    example = li.load(path, sr=sr)[0]
    if example.size < sr:
        example = np.concatenate((example, np.zeros(sr-example.size)))
    else:
        example = example[:sr]
    return example

In [4]:
def get_data(paths, num_threads=4):
    with ThreadPoolExecutor(num_threads) as pool:
        data = list(pool.map(get_audio, paths))
    return np.array(data)

In [5]:
data = get_data(paths, 32)

In [6]:
BATCH_SIZE = 14

train_dataset = tf.data.Dataset.from_tensor_slices(data).shuffle(70).batch(BATCH_SIZE)
train_dataset

<BatchDataset shapes: (None, 16000), types: tf.float64>

In [7]:
discriminator = tf.keras.Sequential([
    tf.keras.layers.InputLayer((16000)),
    tf.keras.layers.Reshape((16000, 1)),
    tf.keras.layers.Conv1D(filters=10, kernel_size=4, strides=1, padding='same', use_bias=False),
    tf.keras.layers.MaxPool1D(4),
    tf.keras.layers.Conv1D(filters=1, kernel_size=4, strides=4, use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.GRU(100),
    tf.keras.layers.Dense(1)
], name='discriminator')

discriminator.summary()

Model: "discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 16000, 1)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 16000, 10)         40        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4000, 10)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 1)           40        
_________________________________________________________________
batch_normalization (BatchNo (None, 1000, 1)           4         
_________________________________________________________________
activation (Activation)      (None, 1000, 1)           0         
_________________________________________________________________
gru (GRU)                    (None, 100)             

In [8]:
generator = tf.keras.Sequential([
    tf.keras.layers.InputLayer((100,)),
    tf.keras.layers.Reshape((100, 1)),
    tf.keras.layers.GRU(100),
    tf.keras.layers.Dense(1000, activation='relu', use_bias=False),
    tf.keras.layers.Reshape((1000, 1)),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=2, strides=1, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=2, strides=2, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=2, strides=2, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=1, strides=1, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=2, strides=2, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=2, strides=2, padding='same', use_bias=False),
    tf.keras.layers.Conv1DTranspose(filters=1, kernel_size=1, strides=1, padding='same', use_bias=False),
    tf.keras.layers.Activation('tanh'),
    tf.keras.layers.Reshape((16000,))
], name='generator')

generator.summary()

Model: "generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 100, 1)            0         
_________________________________________________________________
gru_1 (GRU)                  (None, 100)               30900     
_________________________________________________________________
dense_1 (Dense)              (None, 1000)              100000    
_________________________________________________________________
reshape_2 (Reshape)          (None, 1000, 1)           0         
_________________________________________________________________
conv1d_transpose (Conv1DTran (None, 1000, 1)           2         
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 2000, 1)           2         
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 4000, 1)           2 

In [9]:
latent_dim = 100
noise = tf.random.normal((10, 100))

In [10]:
test = generator(noise)
test.shape

TensorShape([10, 16000])

In [11]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = loss(tf.ones_like(real_output), real_output)
    fake_loss = loss(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def generator_loss(fake_output):
    return loss(tf.ones_like(fake_output), fake_output)

In [12]:
gen_optim = tf.keras.optimizers.Adam(learning_rate=1e-4)
disc_optim = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [13]:
@tf.function()
def train_step(train_batch):
    noise = tf.random.normal([BATCH_SIZE, latent_dim])
    
    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
        generated_audio = generator(noise, training=True)
        
        real_output = discriminator(train_batch, training=True)
        fake_output = discriminator(generated_audio, training=True)
        
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
        
    gen_grad = g_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grad = d_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    gen_optim.apply_gradients(zip(gen_grad, generator.trainable_variables))
    disc_optim.apply_gradients(zip(disc_grad, discriminator.trainable_variables))
    return gen_loss, disc_loss

In [14]:
def train(dataset, epochs):
    start = time.time()
    for i in range(epochs):
        for train_batch in dataset:
            gen_loss, disc_loss = train_step(train_batch)
            
        if (i+1) % 10 == 0:
            print('Epoch {:4d}: gen_loss - {:.4f} | disc_loss - {:.4f} | {:.3f} seconds'.format(
                i+1, gen_loss, disc_loss, time.time()-start))
            start = time.time()
            
        if gen_loss < 0.1 + 0.001:
            break

In [15]:
train(train_dataset, 1000)

Epoch   10: gen_loss - 0.6979 | disc_loss - 1.3747 | 75.043 seconds
Epoch   20: gen_loss - 0.7026 | disc_loss - 1.3578 | 73.003 seconds
Epoch   30: gen_loss - 0.7247 | disc_loss - 0.6993 | 72.151 seconds
Epoch   40: gen_loss - 2.1723 | disc_loss - 0.1230 | 72.721 seconds
Epoch   50: gen_loss - 1.0021 | disc_loss - 0.4850 | 72.806 seconds
Epoch   60: gen_loss - 1.1461 | disc_loss - 2.3429 | 72.669 seconds
Epoch   70: gen_loss - 0.7464 | disc_loss - 1.5140 | 71.307 seconds
Epoch   80: gen_loss - 0.5359 | disc_loss - 0.8827 | 72.817 seconds
Epoch   90: gen_loss - 1.0254 | disc_loss - 0.4467 | 72.385 seconds
Epoch  100: gen_loss - 1.8846 | disc_loss - 0.4117 | 71.409 seconds
Epoch  110: gen_loss - 1.8517 | disc_loss - 0.1731 | 72.289 seconds
Epoch  120: gen_loss - 2.6620 | disc_loss - 0.0745 | 72.202 seconds
Epoch  130: gen_loss - 2.7928 | disc_loss - 0.0653 | 83.586 seconds
Epoch  140: gen_loss - 3.0706 | disc_loss - 0.0495 | 102.068 seconds
Epoch  150: gen_loss - 3.0964 | disc_loss - 0.5

In [16]:
generator.save("models/generative/generators/generator-v0.3.h5")
discriminator.save("models/generative/discriminators/generator-v0.3.h5")

In [22]:
noise = tf.random.normal([BATCH_SIZE, latent_dim])
fake_outputs = discriminator(generator(noise))

In [23]:
generator_loss(fake_outputs)

<tf.Tensor: shape=(), dtype=float32, numpy=0.054438107>

In [25]:
fake_outputs

<tf.Tensor: shape=(14, 1), dtype=float32, numpy=
array([[2.8152895],
       [3.1792119],
       [2.8152776],
       [3.1787128],
       [2.81529  ],
       [2.8152823],
       [2.8153174],
       [3.1791532],
       [2.8153005],
       [2.8152924],
       [2.8152828],
       [2.81532  ],
       [2.815322 ],
       [2.8153014]], dtype=float32)>